# Lorenz 96 problem setup for estimating constant forcing and algorithm testing
By: Rebecca Gjini 

In [ ]:
#Import statements
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numba
from numba import jit, njit
from scipy.linalg import sqrtm
from scipy.optimize import least_squares
import main.EnsembleKalmanAlgorithms as EKA
import main.l96.L96_const_model as gl96
from cycler import cycler

#Plot preferences from palettable.colorbrewer.qualitative import Set1_9
from palettable.colorbrewer.qualitative import Dark2_8

#Plot Preferences 
plt.rcParams['figure.figsize'] = [11.0, 9.0] #size (w, h)
plt.rcParams['figure.dpi'] = 80
plt.rcParams['savefig.dpi'] = 400

plt.rcParams['font.size'] = 18 # controls default text sizes
plt.rcParams['legend.fontsize'] = 'large' # legend fontsize
plt.rcParams['figure.titlesize'] = 'large' # fontsize of the figure title  
plt.rcParams['axes.titlesize'] = 18     # fontsize of the axes title
plt.rcParams['axes.labelsize'] = 32   # fontsize of the x and y labels 35
plt.rcParams['xtick.labelsize'] = 20    # fontsize of the tick labels 17
plt.rcParams['ytick.labelsize'] = 20   # fontsize of the tick labels 17
plt.rcParams['axes.spines.right'] = False #makes right line in plot disappear
plt.rcParams['axes.spines.top'] = False #makes top line in plot disappear
plt.rcParams["font.family"] = "Times New Roman"

plt.rcParams['axes.prop_cycle'] = cycler(color=Dark2_8.mpl_colors)

In [ ]:
file_path = "main/l96/const/data/"
#initialize random seed
np.random.seed(4)

#Creating my sythetic data
#initalize model variables
nx = 40  #dimensions of parameter vector
gamma = [8] #forcing 

t = 0.01  #time step
T_long = 1000  #total time 

#beginning state
int_state = np.random.uniform(0,1,nx)

In [ ]:
#Find the initial condition for my data
spin_up_array = gl96.runge_kutta_v(gamma, int_state, t, T_long)
#intital condition used for the data
x0 = spin_up_array[-1]
# np.savetxt(file_path + "x0.txt", x0, delimiter = ',') 
print(x0)

In [ ]:
#Creating my sythetic data
window = 10
T = 4  + window # total time units, with 4 unit cut off
ny = nx*2   #number of data points

model_out_y_stats = gl96.G(gamma, x0, t, T, np.zeros((nx, nx)))

#Data covariance
T_cov = 8004
RKgam = gl96.runge_kutta_v(gamma, x0, t, T_cov)[int(4/t):] 
gam = np.zeros((ny,int((T_cov - 4)/window)))
for i in range(0, T_cov - 4, window):
    ii = int(i/window)
    gam[:nx,ii] = np.mean(RKgam[int(i/t):int((i+window)/t)], axis = 0)
    gam[nx:,ii] = np.sqrt(np.var(RKgam[int(i/t):int((i+window)/t)], axis = 0, ddof = 1))

R = 2*np.cov(gam) # remove factor of 2 and use mean of samples to get RMSE = 1

R_sqrt_in = EKA.matrix_inv_sqrt(R)
R_sqrt = EKA.matrix_sqrt(R)

#Observations y
y = model_out_y_stats #np.mean(gam, axis = 1)
print(y)

#Solving for initial condition perturbation covariance
covT = 2000  #time to simulate to calculate a covariance matrix of the system
cov_solve = gl96.runge_kutta_v(gamma, x0, t, covT)
ic_cov = 0.1*np.cov(cov_solve.T)
ic_cov_sqrt =  EKA.matrix_sqrt(ic_cov)

# np.savetxt(file_path + "y.txt", y, delimiter = ',')
# np.savetxt(file_path + "R.txt", R, delimiter = ',')
# np.savetxt(file_path + "ic_cov_sqrt.txt", ic_cov_sqrt, delimiter = ',')

In [ ]:
#Intitializing REKI ensemble
Ne_eki = 10        #number of ensemble members

max_runs = 20   #set a maximum number of runs 

nu = 1         # we estimate the forcing

u = np.random.normal(0, 1, size = (nu, Ne_eki))  #try to use uniform distribution
print(u.shape)

# Prior on gamma
mu = np.array([10])
B = np.array([16])
B_sqrt = np.array([4])
# np.savetxt(file_path + "mu.txt", mu, delimiter = ',')
# np.savetxt(file_path + "B.txt", B, delimiter = ',')

In [ ]:
#TEKI Test 
teki_u, teki_f, _ = EKA.TEKI(gl96.G, u, (x0, t, T, ic_cov_sqrt), 
                          y, R, mu, B, min_rmse = 1, method = 'rmse', 
                             tol_x = 1e-4, tol_f = 1e-4, max_iter = max_runs)
print(teki_f)
print(np.mean(teki_u, axis = 1))
ft = gl96.G(np.mean(teki_u, axis = 1), x0, t, T, 0*ic_cov_sqrt)
np.sqrt((np.linalg.norm(R_sqrt_in@(y - ft))**2)/ny)

In [ ]:
etki_u, etki_f, _  = EKA.ETKI(gl96.G, u, (x0, t, T, ic_cov_sqrt), 
                          y, R, mu, B, min_rmse = 1, method = 'rmse', 
                             tol_x = 1e-4, tol_f = 1e-4, max_iter = max_runs)
print(etki_f)
print(np.mean(etki_u, axis = 1))
ft = gl96.G(np.mean(etki_u, axis = 1), x0, t, T, 0*ic_cov_sqrt)
np.sqrt((np.linalg.norm(R_sqrt_in@(y - ft))**2)/ny)

In [ ]:
iekf_u, iekf_f, _  = EKA.IEKF(gl96.G, u, (x0, t, T, ic_cov_sqrt), 
                          y, R, mu, B, min_rmse = 1, method = 'rmse', 
                             tol_x = 1e-4, tol_f = 1e-4, max_iter = max_runs)
print(iekf_f)
print(np.mean(iekf_u, axis = 1))
ft = gl96.G(np.mean(iekf_u, axis = 1), x0, t, T, 0*ic_cov_sqrt)
np.sqrt((np.linalg.norm(R_sqrt_in@(y - ft))**2)/ny)

In [ ]:
uki_u, uki_f, _  = EKA.UKI(gl96.G, (x0, t, T, ic_cov_sqrt), 
                          y, R, mu, B, min_rmse = 1, method = 'rmse', 
                             tol_x = 1e-4, tol_f = 1e-4, max_iter = max_runs)
print(uki_f)
print(np.mean(uki_u, axis = 1))
ft = gl96.G(np.mean(uki_u, axis = 1), x0, t, T, 0*ic_cov_sqrt)
np.sqrt((np.linalg.norm(R_sqrt_in@(y - ft))**2)/ny)

In [ ]:
solution = least_squares(gl96.r, u[:,3], args=(x0, t, T, ic_cov_sqrt, y, R_sqrt_in, mu, B_sqrt), method = 'lm', 
                         xtol = 1e-8, ftol=1e-08)
print(solution.nfev)
print(B_sqrt@solution.x + mu)
print(solution.status)
ft = gl96.G(B_sqrt@solution.x + mu, x0, t, T, 0*ic_cov_sqrt)
print('RMSE:', np.sqrt((np.linalg.norm(R_sqrt_in@(y - ft))**2)/ny))
print(B_sqrt@u[:,3] + mu)

In [ ]:
# Visualizing cost function
gamma_stats_vals = gamma + np.arange(-2, 2, 0.01)

cost_function_stats = np.zeros(len(gamma_stats_vals))
for i in range(0, len(gamma_stats_vals)):
    stats_out = gl96.G([gamma_stats_vals[i]], x0, t, T, ic_cov_sqrt)
    cost_function_stats[i] = np.linalg.norm(R_sqrt_in@(y - stats_out))/np.sqrt(ny)

In [ ]:
plt.figure()
plt.plot(gamma_stats_vals, cost_function_stats)
#stats_truth = gl96.G(gamma, x0, t, T, 0*ic_cov_sqrt)
#plt.scatter(8, 0.5*(np.linalg.norm(R_sqrt_in@(y - stats_truth))**2)/ny, c = 'purple')
plt.title('Cost function')
plt.xlabel('$\\Gamma$')
plt.ylabel('Cost function')
plt.show()